In [4]:
import numpy as np
import pandas as pd
import joblib
import json

#Savepath
save_path = r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\y_randomization_results_HEK.xlsx"

# Load the model
model_base = joblib.load(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\model_LGBM_173f.joblib")

# Load the external data
with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\x\binary\x_ext\x_ext_ecfp_fp_2_1024.json", 'r') as f:
    x_test = json.load(f)

with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\y\binary\y_ext\y_ext_ecfp_fp_2_1024.json", 'r') as f:
    y_test = json.load(f)

# Load the internal data
with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\x\binary\x_train\x_train_ecfp_fp_2_1024.json", 'r') as f:
    x_train = json.load(f)

with open(r"D:\OneDrive\Documentos\LabMol\IC-Citotoxicidade\SERVIDOR CYTO SAFE\New_Cytosafe\HEK293\curated_data\y\binary\y_train\y_train_ecfp_fp_2_1024.json", 'r') as f:
    y_train = json.load(f)



In [5]:
X_ext_infos_df = pd.DataFrame(x_test)
X_ext_infos_df = X_ext_infos_df.add_prefix('var_')

X_int_infos_df = pd.DataFrame(x_train)
X_int_infos_df = X_int_infos_df.add_prefix('var_')


features_sel = np.array(['var_1', 'var_2', 'var_4', 'var_6', 'var_15', 'var_19', 'var_23',
       'var_29', 'var_33', 'var_36', 'var_41', 'var_42', 'var_45',
       'var_46', 'var_63', 'var_64', 'var_72', 'var_74', 'var_80',
       'var_83', 'var_86', 'var_90', 'var_92', 'var_114', 'var_115',
       'var_117', 'var_119', 'var_121', 'var_128', 'var_136', 'var_138',
       'var_139', 'var_140', 'var_175', 'var_191', 'var_197', 'var_209',
       'var_212', 'var_216', 'var_218', 'var_222', 'var_233', 'var_242',
       'var_249', 'var_250', 'var_255', 'var_271', 'var_283', 'var_284',
       'var_288', 'var_294', 'var_301', 'var_310', 'var_314', 'var_316',
       'var_322', 'var_333', 'var_352', 'var_361', 'var_367', 'var_378',
       'var_381', 'var_383', 'var_387', 'var_389', 'var_393', 'var_407',
       'var_418', 'var_428', 'var_452', 'var_456', 'var_458', 'var_472',
       'var_480', 'var_481', 'var_482', 'var_484', 'var_486', 'var_491',
       'var_492', 'var_511', 'var_518', 'var_519', 'var_521', 'var_523',
       'var_531', 'var_540', 'var_549', 'var_551', 'var_556', 'var_561',
       'var_562', 'var_573', 'var_574', 'var_579', 'var_585', 'var_590',
       'var_602', 'var_628', 'var_641', 'var_646', 'var_650', 'var_651',
       'var_652', 'var_656', 'var_659', 'var_665', 'var_667', 'var_669',
       'var_673', 'var_675', 'var_689', 'var_692', 'var_695', 'var_698',
       'var_699', 'var_703', 'var_713', 'var_714', 'var_715', 'var_718',
       'var_726', 'var_730', 'var_736', 'var_738', 'var_739', 'var_741',
       'var_751', 'var_758', 'var_767', 'var_772', 'var_778', 'var_785',
       'var_792', 'var_798', 'var_799', 'var_801', 'var_802', 'var_807',
       'var_828', 'var_831', 'var_832', 'var_841', 'var_842', 'var_843',
       'var_847', 'var_849', 'var_852', 'var_864', 'var_875', 'var_881',
       'var_883', 'var_886', 'var_890', 'var_893', 'var_896', 'var_899',
       'var_904', 'var_922', 'var_926', 'var_932', 'var_935', 'var_940',
       'var_946', 'var_950', 'var_976', 'var_980', 'var_985', 'var_998',
       'var_1005', 'var_1008', 'var_1017', 'var_1019'], dtype=object)


In [6]:
features = sorted(features_sel, reverse = False, key = lambda ele: int(ele.split('_')[1]))
x_ext_new = X_ext_infos_df[features]
x_int_new = X_int_infos_df[features]

In [7]:
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, f1_score
import lightgbm as lgb
import pandas as pd
import numpy as np

params = model_base.get_params()

# Definir o número de rounds de y-randomization
n_rounds = 50

# Inicializar dataframe para armazenar os resultados
resultados = pd.DataFrame(columns=['Round', 'Balanced Accuracy', 'MCC', 'F1 Score'])

# Realizar os rounds de y-randomization
for i in range(n_rounds):
    # Embaralhar as variáveis dependentes no dataset de treino e ajustar o modelo
    y_teste_random = np.random.permutation(y_train)

    model_y = lgb.LGBMClassifier(**params)
    model_y.fit(x_train, y_teste_random)

    y_pred = model_y.predict(x_test)

    # Calcular as métricas de desempenho
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Apender os resultados ao dataframe
    resultados.loc[i] = [i+1, balanced_accuracy, mcc, f1]

# Exibir o dataframe com os resultados
resultados.to_excel(save_path, index=False)
print(resultados)

c:\Users\LabMo\anaconda3\envs\my-rdkit-env\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 4913, number of negative: 24563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.083732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2048
[LightGBM] [Info] Number of data points in the train set: 29476, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166678 -> initscore=-1.609356
[LightGBM] [Info] Start training from score -1.609356


c:\Users\LabMo\anaconda3\envs\my-rdkit-env\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 4913, number of negative: 24563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.084593 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2048
[LightGBM] [Info] Number of data points in the train set: 29476, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166678 -> initscore=-1.609356
[LightGBM] [Info] Start training from score -1.609356


c:\Users\LabMo\anaconda3\envs\my-rdkit-env\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Info] Number of positive: 4913, number of negative: 24563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077518 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2048
[LightGBM] [Info] Number of data points in the train set: 29476, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166678 -> initscore=-1.609356
[LightGBM] [Info] Start training from score -1.609356


c:\Users\LabMo\anaconda3\envs\my-rdkit-env\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 4913, number of negative: 24563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082803 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2048
[LightGBM] [Info] Number of data points in the train set: 29476, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166678 -> initscore=-1.609356
[LightGBM] [Info] Start training from score -1.609356


c:\Users\LabMo\anaconda3\envs\my-rdkit-env\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 4913, number of negative: 24563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2048
[LightGBM] [Info] Number of data points in the train set: 29476, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166678 -> initscore=-1.609356
[LightGBM] [Info] Start training from score -1.609356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
    Round  Balanced Accuracy       MCC  F1 Score
0     1.0           0.510189  0.030396  0.756344
1     2.0           0.506036  0.018484  0.754369
2     3.0           0.497973 -0.006597  0.750431
3     4.0           0.501883  0.005854  0.752008
4     5.0           0.497566 -0.007932  0.750179
5     6.0           0.497160 -0.008740  0.748602
6     7.0           0.5